In [1]:
import sys
sys.executable

'/home/yaoms/anaconda3/envs/PytorchEnv/bin/python'

In [1]:
import sys
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import os
import cv2
from torch.utils.data import DataLoader, TensorDataset, Dataset

%load_ext autoreload
%autoreload 2
# %matplotlib inline
sys.path.append("/home/shiyi/gpu/")

from gpu_allocation import set_gpu
num_gpu = 1
set_gpu(num_gpu)


Found 8 GPU(s) only 6 gpu below threshold
Using GPU 3


In [2]:
#hyperparameters 

IMG_SIZE = 224
BATCH_SIZE = 128
CLASSES = 8
EPOCH = 200

IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LR = 0.001

In [3]:
#dir and path

csv_path = '/data/AlgProj/tct_yaoms/data/tct_0702_preprocess_iou0.1_img_series/iou0.1dataaumentation.csv'
csv = pd.read_csv(csv_path)


In [4]:
#train test dataset split
#train val transforms definition

def train_test_split(df, test_size, shuffle=True):
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)
    #split
    length = len(df)
    threspoint = int((1 - test_size)*length)
    train_df = df.loc[:threspoint-1,:]
    test_df = df.loc[threspoint:,:]
    
    d = {}
    d['train'] = train_df
    d['test'] = test_df.reset_index(drop=True)
    return d

train_transformer = transforms.Compose([transforms.Resize((256,256)),
                                 transforms.RandomRotation((-180, 180)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomResizedCrop((224,224)),  
                                 transforms.ToTensor(),
                                       transforms.Normalize(IMG_MEAN,IMG_STD)]
                                )
test_transformer = transforms.Compose([transforms.Resize((256,256)),
                                  transforms.CenterCrop((224,224)),
                                 transforms.ToTensor(),
                                      transforms.Normalize(IMG_MEAN,IMG_STD)])
train_test_transformer = {'train':train_transformer, 'test':test_transformer}

In [5]:
train_test_dict = train_test_split(csv, 0.2)
len(train_test_dict['test'])

787

In [6]:
class TCT_Dataset(Dataset):
    def __init__(self, phase, transforms=True):
        self.phase = phase
        self.transforms = transforms
        if phase == 'train':
            self.df = train_test_dict['train']
        else:
            self.df = train_test_dict['test']
        
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img_path = self.df.loc[idx, 'img_path']
        img = Image.open(img_path).convert('RGB')
        label = self.df.loc[idx, 'label']
        
        if self.transforms:
            if self.phase == 'train':
                transformer = train_test_transformer['train']
            else:
                transformer = train_test_transformer['test']

            img = transformer(img)
        
        return (img, label)

In [7]:
train_dataset = TCT_Dataset('train')
test_dataset = TCT_Dataset('test')

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [8]:
from torchvision.models import resnet18
model = resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

fc_features = model.fc.in_features
model.fc = torch.nn.Linear(fc_features, CLASSES)

model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [17]:
optimizer = optim.SGD([{'params':model.fc.parameters(), 'lr':0.0001}])
#                        {'params':model.layer4.parameters(),'lr':0.00001}
criterion = nn.CrossEntropyLoss()

In [14]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    print ('Train Epoch: {}\t Loss: {:.6f}\n'.format(epoch,loss.item()))

    
def test(model, device, test_loader, optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i,data in enumerate(test_loader):          
            x,y= data
            x=x.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            test_loss = criterion(y_hat, y).item() # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
#     test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(train_test_dict['test'])
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(train_test_dict['test']),
        acc))
    
#     print('-----------------------')
    return acc

In [19]:
max_acc = 0
max_epoch = 0
for epoch in range(EPOCH):
    train(model, DEVICE, train_loader, optimizer, epoch)
    acc = test(model, DEVICE, test_loader, optimizer)
    if acc >= max_acc:
        max_acc = acc
        max_epoch = epoch
    print('current best acc: ', max_acc)
    print('-----------------------')
print('Result: at {} epoch, achieved best acc: {:.0f}'.format(max_epoch, max_acc))

Train Epoch: 0	 Loss: 0.585691


Test set: Average loss: 0.5835, Accuracy: 624/787 (79%)

current best acc:  79.28843710292249
-----------------------
Train Epoch: 1	 Loss: 0.568544


Test set: Average loss: 0.5752, Accuracy: 624/787 (79%)

current best acc:  79.28843710292249
-----------------------
Train Epoch: 2	 Loss: 0.615351


Test set: Average loss: 0.6927, Accuracy: 623/787 (79%)

current best acc:  79.28843710292249
-----------------------
Train Epoch: 3	 Loss: 0.859049


Test set: Average loss: 0.4966, Accuracy: 620/787 (79%)

current best acc:  79.28843710292249
-----------------------
Train Epoch: 4	 Loss: 0.643151


Test set: Average loss: 0.3642, Accuracy: 621/787 (79%)

current best acc:  79.28843710292249
-----------------------
Train Epoch: 5	 Loss: 0.645198


Test set: Average loss: 0.2602, Accuracy: 621/787 (79%)

current best acc:  79.28843710292249
-----------------------
Train Epoch: 6	 Loss: 0.625772


Test set: Average loss: 0.6353, Accuracy: 619/787 (79%)

curr

Train Epoch: 54	 Loss: 0.583281


Test set: Average loss: 0.4126, Accuracy: 624/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 55	 Loss: 0.620733


Test set: Average loss: 0.5257, Accuracy: 623/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 56	 Loss: 0.820076


Test set: Average loss: 0.6041, Accuracy: 620/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 57	 Loss: 0.820197


Test set: Average loss: 0.4222, Accuracy: 617/787 (78%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 58	 Loss: 0.638248


Test set: Average loss: 0.5741, Accuracy: 620/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 59	 Loss: 0.687803


Test set: Average loss: 0.7072, Accuracy: 618/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 60	 Loss: 0.564563


Test set: Average loss: 0.3608, Accuracy: 624/787 (79%

Train Epoch: 108	 Loss: 0.584646


Test set: Average loss: 0.6382, Accuracy: 619/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 109	 Loss: 0.640455


Test set: Average loss: 0.5001, Accuracy: 624/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 110	 Loss: 0.524312


Test set: Average loss: 0.4570, Accuracy: 627/787 (80%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 111	 Loss: 0.588234


Test set: Average loss: 0.7035, Accuracy: 621/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 112	 Loss: 0.629767


Test set: Average loss: 0.3796, Accuracy: 620/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 113	 Loss: 0.653214


Test set: Average loss: 0.7348, Accuracy: 622/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 114	 Loss: 0.625042


Test set: Average loss: 0.3646, Accuracy: 621/7

Train Epoch: 162	 Loss: 0.680448


Test set: Average loss: 0.7179, Accuracy: 617/787 (78%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 163	 Loss: 0.672329


Test set: Average loss: 0.7309, Accuracy: 619/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 164	 Loss: 0.573103


Test set: Average loss: 0.3639, Accuracy: 619/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 165	 Loss: 0.630900


Test set: Average loss: 0.3446, Accuracy: 623/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 166	 Loss: 0.534728


Test set: Average loss: 0.3947, Accuracy: 622/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 167	 Loss: 0.628071


Test set: Average loss: 0.4536, Accuracy: 621/787 (79%)

current best acc:  79.66963151207116
-----------------------
Train Epoch: 168	 Loss: 0.639007


Test set: Average loss: 0.4051, Accuracy: 622/7